In [ ]:
import pandas as pd
import numpy
import openpyxl
import re

In [ ]:
# data1 = pd.read_excel("CDOT/CDOTRM_CD_Crash_Listing_-_2015.xlsx", parse_dates=[['date', 'time']])
# data2 = pd.read_excel("CDOT/CDOTRM_CD_Crash_Listing_-_2016.xlsx", parse_dates=[['date', 'time']])
# data3 = pd.read_excel("CDOT/CDOTRM_CD_Crash_Listing_-_2017.xlsx", parse_dates=[['date', 'time']])
# data4 = pd.read_excel("CDOT/CDOTRM_CD_Crash_Listing_-_2018.xlsx", parse_dates=[['date', 'time']])
data5 = pd.read_excel("CDOT/CDOTRM_CD_Crash_Listing_-_2019.xlsx", parse_dates=[['date', 'time']])

In [ ]:
# data = pd.concat([data1, data2, data3, data4, data5])
data = data5

In [ ]:
crash_data = data[data['county'] == 'EL PASO']
crash_data = crash_data[['date_time', 'loc_01', 'severity', 'condition', 'lighting']]

crash_data = crash_data.dropna()

crash_data['date_time'] = pd.to_datetime(crash_data['date_time'])

crash_data = crash_data.reset_index(drop=True)
crash_data['date_time'] = crash_data['date_time'].dt.round('H')

In [ ]:
for i in range(len(crash_data)):
    curr_street = crash_data['loc_01'].get(i)
    # Get rid of the parentheses
    curr_street = curr_street.split('(')[0]

    curr_street_list = curr_street.split(' ')
    if curr_street_list[-1] == '':
        curr_street_list = curr_street_list[:-1]

    # Change the highway format to match with Waze data
    if curr_street_list[0] == 'HWY':
        curr_street = 'SH-' + curr_street_list[1]

    # Move the ESWN indicator to the front to match with Waze data
    else:
        pattern = '^[ESWN]$'
        if re.match(pattern, curr_street_list[-1]):
            curr_street = curr_street_list[-1] + ' ' + ' '.join(curr_street_list[:-1])

    # Deleting the ending numbers
    if curr_street.split(' ')[-1].isnumeric():
        curr_street = ' '.join(curr_street.split(' ')[:-1])

    crash_data.at[i, 'loc_01'] = curr_street

In [ ]:
print(crash_data)
crash_data.to_csv("CDOT_cleaned.csv")

                date_time         loc_01 severity condition      lighting
0     2019-01-28 12:00:00  11200 HWY 83       INJ     SNOWY      DAYLIGHT
1     2019-07-01 01:00:00      S 13TH ST      PDO       DRY  DARK-LIGHTED
2     2019-07-08 18:00:00      S 14TH ST      PDO       DRY      DAYLIGHT
3     2019-12-17 16:00:00      S 14TH ST      INJ       ICY      DAYLIGHT
4     2019-06-17 07:00:00      S 15TH ST      PDO       DRY      DAYLIGHT
...                   ...            ...      ...       ...           ...
12111 2019-10-09 23:00:00   S NEVADA AVE      INJ       DRY  DARK-LIGHTED
12112 2019-09-21 20:00:00   S NEVADA AVE      PDO       DRY  DARK-LIGHTED
12113 2019-02-28 10:00:00   S NEVADA AVE      PDO       DRY      DAYLIGHT
12114 2019-09-13 02:00:00   S NEVADA AVE      PDO       DRY  DARK-LIGHTED
12115 2019-09-06 11:00:00   S NEVADA AVE      PDO       DRY      DAYLIGHT

[12116 rows x 5 columns]
